In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pystan
import pandas as pd
import networkx as nx

plt.style.use('ggplot')
%matplotlib inline

## Latent space model
One way to model network data is to assume that each node has a latent position in $D$-dimensional Euclidean space and that the rates are dependent on the relative distance between he nodes in this latent space. This is the framework of Hoff, Raftery and Handcock in [2]. As the weights in the network (that is, the observations), we model the likelihood as iid. Poisson variables whose rates depend on the relative distance between the nodes in the latent space as is done [here](http://edwardlib.org/tutorials/latent-space-models).

The latent positions are drawn from an isotropic multivariate Gaussian with 0 mean and diagonal covariance, $\lambda I$ where $I$ denotes the identity matrix of appropriate size (ie. $D\times D$). The variance parameter $\lambda$ is drawn from a noninformative uniform prior (improper from 0 to $\infty$).

The likelihood of the modle is then as such,

$$ p(X|\theta) = \prod_{i\neq j} p(x_{i,j} | \boldsymbol{z}_i,\boldsymbol{z}_j)$$

where $\boldsymbol{z}_n$ denotes the $D$-dimensional latent position.  
Each term in the likelihood is Poisson, with rate depending on the positions,
$$p(x_{i,j} | \boldsymbol{z}_i,\boldsymbol{z}_j) = Poisson\left(\frac{1}{\| \boldsymbol{z}_i - \boldsymbol{z}_j\|}\right)$$

such that two positions that are close to eachother in latent space has a higher Poisson rate and on average, we'll observe a higher weight between them (in this framework, this corresponds to more interactions between different characters in Game of Thrones). $\|\cdot\|$ denotes the Euclidean 2-norm such that $\|x-y\|$ is the Euclidean distance between $x$ and $y$.


The generative model is then as such:

 1. Draw $\lambda$ from a non-informative improper prior (this is done by not explicitly defining a prior in STAN).
 2. for $n = 1..N$
     * draw $\boldsymbol{z}_n \sim \mathcal{N}(0, \lambda I)$ where $I$ is of size $D \times D$
 3. for all pairs $i,j\in \{1,2,\ldots,N\}\times\{1,2,\ldots,N\}$ where $i\neq j$
     * draw $x_{i,j} \sim Poisson\left(\frac{1}{\| \boldsymbol{z}_i - \boldsymbol{z}_j\|}\right)$
     

From this we get the factorization of the joint probabiltiy:

$$p(X, \boldsymbol{z}, \lambda) = \prod_{i\neq j} [p(x_{i,j} | \boldsymbol{z}_i, \boldsymbol{z}_j)] \prod_{n=1}^N [p(\boldsymbol{z}_n | 0, \lambda)] p(\lambda)$$

and the corresponding PGM

![PGM2](https://i.imgur.com/RumjQEV.png)

which of course is very simple.

Below the model is implemented in STAN


In [4]:
# see http://edwardlib.org/tutorials/latent-space-models

model_code = """
data { 
    int<lower=2> N; // number of nodes
    int<lower=1> D; // dimension of latent space
    int<lower=0> X[N,N]; // adjacency matrix
} parameters { 
    matrix[N,D] z; // latent positions in space
    real<lower=0> lambda; // std
} model {

    for (d in 1:D) z[:,d] ~ normal(0, lambda);
    for (i in 1:N){
        for (j in 1:N) {
            if (i == j) continue;
            X[i,j] ~ poisson(1.0 / (distance(z[i,:], z[j,:]) + 1e-6)); // can either be this or maybe poisson(exp(alpha-dist)) where alpha ~ normal(0, sigma) where lambda is small
        }
    }
    
}
"""



In [5]:
sm = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_271a92d5bbb05c03435b49986fc4caff NOW.
/home/david/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmp6wmh2_hf/stanfit4anon_model_271a92d5bbb05c03435b49986fc4caff_5221252646562349184.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


### Test on Synthetic data
We generate a small network and test the model here. 

In [6]:
# Generate small network to test on
num_each = 50
G = nx.barbell_graph(num_each,0)

# Generate the notes with different number of edges.
mat = nx.to_numpy_array(G, dtype=int)
rate1 = 5
rate2 = 100
mat1 = np.random.poisson(rate1, size=(num_each//2, num_each//2))
mat1 = np.tril(mat1) + np.tril(mat1,k=-1).T
mat2 = np.random.poisson(rate2, size=mat1.shape)
mat2 = np.tril(mat2) + np.tril(mat2, k=-1).T

mat = np.array(np.block([[mat1, np.zeros(mat1.shape, dtype=int)], [np.zeros(mat1.shape, dtype=int), mat2]]))
np.fill_diagonal(mat, 0)
mat[0,-1] = mat[-1,0] = np.random.poisson(8)

# Draw graph
G = nx.from_numpy_array(mat)
nx.draw(G)

/home/david/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


Here we see two very distinct clusters. This means we should see two similar clusterings in the two-dimensional Euclidean space. The data is generated by generating two independent Poisson 'blocks' and generating the network accordingly. In this case, we have two very different rates, so the two clusters are very distict.

In [8]:
data = {'X' : mat, 'N' : mat.shape[0], 'D' : 2}
fit = sm.sampling(data, chains=1, iter=10000) 

### EXTREMELY SLOW FOR THE DATA ABOVE


There is a problem¸however. Compared to a Stochastic Block Model (see below), we see that the clusters are not modelled explicitly. This is a problem if the in-group rates are very different like here (100 for one group and 5 for the other in one of our experiments). In this case, the group with the higher rate should be put very close together, which may happen at the cost of the other group positions due to the shared variance parameter $\lambda$. The inference process was very slow in this case, and we gave it up due to time constraints.

In [6]:
num_each = 50
G = nx.barbell_graph(num_each,0)

# Generate the notes with different number of edges.
mat = nx.to_numpy_array(G, dtype=int)
rate1 = 5
rate2 = 8
mat1 = np.random.poisson(rate1, size=(num_each//2, num_each//2))
mat1 = np.tril(mat1) + np.tril(mat1,k=-1).T
mat2 = np.random.poisson(rate2, size=mat1.shape)
mat2 = np.tril(mat2) + np.tril(mat2, k=-1).T

mat = np.array(np.block([[mat1, np.zeros(mat1.shape, dtype=int)], [np.zeros(mat1.shape, dtype=int), mat2]]))
np.fill_diagonal(mat, 0)
mat[0,-1] = mat[-1,0] = np.random.poisson(8)

# Draw graph
G = nx.from_numpy_array(mat)
nx.draw(G)

In [ ]:
#data = {'X' : mat, 'N' : mat.shape[0], 'D' : 2}
#fit = sm.sampling(data, chains=1, iter=10000) 

## ALSO EXTREMELY SLOW

Setting the parameters close to eachother was also very slow. This was probably due to the shared variance parameter $\lambda$. This makes the model relatively unflexible, as the only thing influencing the rate is the inverse distance. If the distance required to model the data well (ie. get high likelihood) is very different in different parts of the network, we are constraining ourselves by having all latent positions $\boldsymbol{z}_n$ share variances. This results in many samples being rejected by NUTS, and so slow sampling. A way to fix this, could be to model the groups explicitly.  
This is exactly the case in the Stochastic Block Model introduced below.

A note should be made regarding the general perfomance of the latent space approach. As noted in [2], this approach works suprisingly well for binary networks. In this case, the likelihood becomes an 'augmented Bernoulli-logistic' framework, where we say that the probability of observing a link is like in Bayesian logistic regression where the distance between the latent positions is one of the features in the logistic regression. This also allows for easy inclusion of covariate information of the nodes. Perhaps we could implement a similar frameowrk here, as Poisson regression, but we have chosen to work with the Stochastic Block Model framework instead.

---

## Stochastic Block Model with Poisson/Gamma observations


We are trying to do analysis on a weighted (positive integer weights), undirected network on interactions between characters in Game of Thrones using a Stochastic Block Model [1]


### Modelling
#### Theory
As mentioned above we will fit a Stochastic Block Model (SBM) to the Game of Thrones data. The SBM is a mixture model where we assume that each node belongs to a single mixture component. Since each edge connects two nodes its likelihood will depend on the corresponding two mixture components. We let $N$ denote the number of nodes and $K$ denote the number of mixture components

We assume that the edges are i.i.d and hence get that the likelihood of the network takes the form:
$$ p(X|\theta) = \prod_{i\neq j} p(x_{i,j}|z_i, z_j, \phi) $$
Where $\theta = \{ \{z_i \}_{i=1}^N, \phi\}$. Here $z_i$ indicates which mixture component the $i$th node belongs to and $\phi$ denotes any further parameters depending on the distribution. 
A simple choice of distribution would be a Poisson distribution. Then each probability in the product would be
$$ p(x_{i,j} | z_i, z_j, \lambda) = Poisson(\lambda_{z_i,z_j})  $$
Where $\lambda$ is a matrix with the rates between groups s.t. $\lambda_{i,j}$ is the rate between rate $i$ and $j$. We choose to assume that $\lambda_{i,j} \sim Gamma(a,b)$ where $a$ and $b$ are chosen hyperparameters.
Next we assume the node mixture indicators $z$ to come from a $K$-dimensional categorical distribution
$$ p(z_i = k | \pi) = \pi_k  $$
where $\pi = \{ \pi_k\}_{k=1}^K$ and $\sum_{k=1}^k \pi_k = 1$. We assume that $\pi$ is Dirichlet distributed with prior $\alpha$

From the above modelling we get the following generative process:
* Given parameters $a$, $b$ and $\alpha$
    1. draw $\pi \sim Dirichlet(\alpha)$
    2. for $n = 1..N$
        * draw $z_i \sim Categorical(\pi)$
    3. draw $\lambda \sim Gamma(a,b)$
    4. for all $i,j$ where $i \neq j$
        * draw $x_{i,j} \sim Poisson(\lambda_{z_i,z_j})$

Which can also be depicted in the a PGM:

![PGM1](https://i.imgur.com/tLcH5KT.png)

However as STAN does not do well with discrete latent variables we cannot directly infer $z$ in the above model. This is a well known shortcoming STAN well documented in chapter 5 of the 'STAN User's Guide'. The work around is marginalizing out the discrete variables in the likelihood by summing:

$$ p(X_{i,j}|\lambda) = \sum_{k=1}^K \sum_{k'=1}^K \text{Categorical}(z_i = k|\lambda)\text{Categorical}(z_j = k'|\lambda)\text{Poisson}(X_{ij}| \lambda_{k,k'})    $$

In the implementation in STAN we transfer the above to the log scale for numerical reasons.

Below is an implementation in STAN where we infer $\lambda$ in the above model using the marginalization work around

### STAN model

In [9]:
model_code= """
data{
       int<lower=1> N;                    // number of nodes
        int X[N,N];                       // adjacency matrix
        int<lower=2> K;                   // number of groups
        vector[K] alpha;                  // prior for dirichlet distribution
        real a;                           // priors on lambda
        real b;
} parameters {
        simplex[K] pi;                     // group sizes
        vector<lower=0>[K*(K+1)/2] lambda; // rates between groups
} model{
        matrix[K,K] log_mix_prop;          // log mixing proportions,
        matrix[K,K] lps;                   

        matrix[K,K] tmp = rep_matrix(0, K, K); // temp varible
        
        int count = 1;                    // variables used to ensure symmetry of lambda
        int len = K*(K+1)/2;
        
        pi ~ dirichlet(alpha); // prior on group sizes,
        
        for(k in 1:len){
            lambda[k] ~ gamma(a, b);
        }
        
        for(k1 in 1:K){
            for(k2 in k1:K){
                    tmp[k1, k2] = lambda[count];
                    count = count+1;
            }
        }
        tmp = tmp + tmp' - diag_matrix(diagonal(tmp));
        
        // cache mixing proportions
        for (k1 in 1:K) {
            for (k2 in 1:K) {
                log_mix_prop[k1, k2] = log(pi[k1]) + log(pi[k2]);
            }
        }
        
        // likelihood,
        for (i in 1:N) {
            for (j in 1:N) {
                if (i == j) continue; // ignore diagonal
                lps = log_mix_prop; // reset log probs
                for (k1 in 1:K) {
                    for (k2 in 1:K) {
                        lps[k1, k2] += poisson_lpmf(X[i,j] | tmp[k1,k2]);
                    }
                }
                target += log_sum_exp(lps);
            }
        }
}
"""

In [23]:
sm = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c4c3dce24cad2da7f3baba60dce61707 NOW.


## Test on synthetic data
In order to test our model we generate a simple network strcture to test on

In [24]:
# Generate small network to test on
num_each = 50
G = nx.barbell_graph(num_each,0)

# Generate the notes with different number of edges.
mat = nx.to_numpy_array(G, dtype=int)
rate1 = 5
rate2 = 100
mat1 = np.random.poisson(rate1, size=(num_each//2, num_each//2))
mat1 = np.tril(mat1) + np.tril(mat1,k=-1).T
mat2 = np.random.poisson(rate2, size=mat1.shape)
mat2 = np.tril(mat2) + np.tril(mat2, k=-1).T

mat = np.array(np.block([[mat1, np.zeros(mat1.shape, dtype=int)], [np.zeros(mat1.shape, dtype=int), mat2]]))
np.fill_diagonal(mat, 0)
mat[0,-1] = mat[-1,0] = np.random.poisson(8)

# Draw graph
G = nx.from_numpy_array(mat)
nx.draw(G)

This will be the test graph. Cleary we can see the two mixture components an their different rates.

In [25]:
data = {'X' : mat, 'N' : mat.shape[0], 'K' : 2, 'alpha' : np.ones(2), 'a' : 7.5, 'b' : 1}
fit = sm.sampling(data=data, iter=10000, chains=1)

In [27]:
fit

Inference for Stan model: anon_model_c4c3dce24cad2da7f3baba60dce61707.
1 chains, each with iter=10000; warmup=5000; thin=1; 
post-warmup draws per chain=5000, total post-warmup draws=5000.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
pi[1]        0.5  1.1e-4 7.1e-3   0.48   0.49    0.5    0.5   0.51   4219    1.0
pi[2]        0.5  1.1e-4 7.1e-3   0.49    0.5    0.5   0.51   0.52   4219    1.0
lambda[1]   5.04  1.4e-3    0.1   4.86   4.98   5.04    5.1   5.23   5010    1.0
lambda[2]   0.01  5.3e-5 3.6e-3 5.0e-3 8.4e-3   0.01   0.01   0.02   4467    1.0
lambda[3] 100.37  5.9e-3   0.41  99.58 100.09 100.38 100.64 101.15   4736    1.0
lp__       -6125    0.03   1.44  -6129  -6126  -6125  -6124  -6123   2514    1.0

Samples were drawn using NUTS at Tue Apr 23 10:54:27 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

Examining the fit we see that the different mixtures aswell as the different rates between groups. 

## Test on real data
### Data import
Import of the datafiles. The contents of the dataset can be seen in the intial initial_notebook.

In short the data concist of an undireced netwwork between persons in the TV series games of thrones. 

We want to infer communities in the network based on the observed edges in the network. The observed data is represented through the adjacency matrix of the network of size $N \times N$ such that N is the observed number of nodes (ie. characters). Then $X[i,j] = w$ is the observed number of interactions, w, between character i and j. $X$ is symmetric, as the network is undirected.

In [50]:
# load nodes and edges from file
nodes = pd.read_csv('../data/got-s6-nodes.csv')
edges = pd.read_csv('../data/got-s6-edges.csv')

# Initialize the graph based on the data from Game of Thrones Season 6
G = nx.Graph(name='Game Of Thrones S06')
G.add_nodes_from(nodes['Id'])
edge_dict = [(x['Target'],x['Source'], {'weight' : x['Weight']}) for (_,x) in edges.iterrows()]
G.add_edges_from(edge_dict)

# Conver to numpy to use with STAN
graph = nx.to_numpy_matrix(G, dtype=int)
nx.draw(G)

The season 6 graph also concist of distinct grahps. So initially we se the mixturinging components to be 2

In [48]:
n = len(nodes)+1
k = 2
data = {'X' : graph, 'N' : n, 'K' : k, 'alpha' : np.ones(2), 'a' : 7.5, 'b' : 1}
fit = sm.sampling(data=data, iter=500, chains=1)

In [51]:
fit

Inference for Stan model: anon_model_c4c3dce24cad2da7f3baba60dce61707.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
pi[1]       0.21  2.7e-4 3.8e-3    0.2    0.2   0.21   0.21   0.21    195    1.0
pi[2]       0.79  2.7e-4 3.8e-3   0.79   0.79   0.79    0.8    0.8    195    1.0
lambda[1]  13.85    0.03   0.31  13.31  13.63  13.83  14.09  14.43    109    1.0
lambda[2] 1.1e-3  3.3e-5 3.5e-4 5.8e-4 8.7e-4 1.1e-3 1.3e-3 2.0e-3    113    1.0
lambda[3]   0.05  4.8e-4 5.2e-3   0.04   0.05   0.05   0.06   0.06    116    1.0
lp__      -1.3e4    0.13   1.14 -1.3e4 -1.3e4 -1.3e4 -1.3e4 -1.3e4     73    1.0

Samples were drawn using NUTS at Tue Apr 23 11:16:12 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

it works

## Current problems
1. We're working on extracting the mixture component $z_{ij}$ to determine the community in which a egde belongs to
2. We're having issues setting up the $\lambda$'s such that we allow for a node to have a belonging to multiple clusters. From the intial fit on the synthetic data we saw that we found the right rates. however they're not always located in the diagonal of the matrix $\lambda$.

## Litterature
[1] Mikkel N. Schmidt and Morten Mørup. Non-parametric bayesian modeling of complex networks. an introduction. IEEE Signal Processing Magazine, 30(3):110–128, May 2013. [PDF](http://mikkelschmidt.dk/papers/schmidt2013spm.pdf) , [Link to publication](https://ieeexplore.ieee.org/document/6494690)  
[2] Peter D Hoff, Adrian E Raftery & Mark S Handcock (2002) Latent Space Approaches to Social Network Analysis, Journal of the American Statistical Association, 97:460, 1090-1098, DOI: 10.1198/016214502388618906 